# 네이버 영화 리뷰

직접 크롤링하기엔 양이 많으니 https://github.com/e9t/nsmc 에 있는 corpus를 사용한다.

#### 파일 내용 구경하기

```bash
!head ../data/ratings.txt
```

#### CSV 파일 불러와서 긍정/부정 리뷰 나누기
```python
import csv

pos_review = []
neg_review = []

with open('../data/ratings.txt', 'r') as data_file:
    reader = csv.reader(data_file, delimiter='\t')
    header = next(reader)
    for row in reader:
        if int(row[2]) == 1:
            pos_review.append(row[1])
        else:
            neg_review.append(row[1])
```

#### 1000개씩 랜덤 샘플링하기
```python
import numpy as np

samples = np.random.choice(len(pos_review), 1000,
                           replace=False)
samples.sort()

pos_review = [pos_review[i] for i in samples]
neg_review = [neg_review[i] for i in samples]
```

#### 형태소 분석해서 형용사 숫자 세기
```python
from konlpy.tag import Twitter
from collections import Counter

def count_adjs(reviews):
    stopwords = ['있다', '없다', '이렇다', '같다', '아니다', '스럽다', '이다']
    
    twt = Twitter()
    counts = Counter()
    
    for review in reviews:
        tags = twt.pos(review, stem=True)
        for morph, tag in tags:
            if tag == 'Adjective' and morph not in stopwords:
                counts[morph] += 1
    
    return counts
   
pos_counts = count_adjs(pos_review)
neg_counts = count_adjs(neg_review)
```

## 시각화

### 1. 워드클라우드 그리기
```python
%matplotlib inline
import matplotlib.pyplot as plt
from wordcloud import WordCloud

cloud = WordCloud(width=900, height=600, 
                  font_path='../data/08서울남산체 B.ttf', 
                  background_color='white')
cloud = cloud.fit_words(pos_counts.items())

plt.figure(figsize=(15, 20))
plt.axis('off')
plt.imshow(cloud)
plt.show()
```

### 2. Bar Plot 그리기

#### Matplotlib 한글 폰트 확인
Seaborn은 영문 폰트를 기본으로 사용하기 때문에 한글은 네모네모로 깨져나온다. 한글 폰트를 확인하고 Seaborn을 세팅할 때 사용한다.
```python
import matplotlib
matplotlib.font_manager.OSXInstalledFonts()
```

#### Seaborn import 및 세팅
```python
import seaborn as sb

sb.set(style='whitegrid', font='AppleGothic')
sb.set_color_codes('pastel')
```

#### Plot에 나타낼 값 설정
긍정 리뷰에서 나타난 형용사들 중 상위 20개를 그려본다. 먼저 형용사를 세어 둔 `pos_counts` 딕셔너리의 아이템들을 가져온 다음, 많이 등장한 형용사부터 내림차순으로 정렬한다.
```python
from operator import itemgetter
items = pos_counts.items()
items = sorted(items, key=itemgetter(1), reverse=True)
items = items[:20]
x = [count for tag, count in items]
y = [tag for tag, count in items]
```

#### Bar Plot 그리기
```python
plt.figure(figsize=(10, 10))
sb.barplot(x=x, y=y, color='b')
```